# Imports and basic set up

In [116]:
#In the begining, ther is time
import time

In [117]:
#Quality of life
#Some things we do generate a lot of warnings, and it just becomes clutter.
import warnings
warnings.filterwarnings("ignore")
#I like it when my notebook helps me out
%config IPCompleter.greedy=True
#Sometiems you just need to print pretty
from pprint import pprint
#Lots of these operations take many minutes to complete. So it behooves us to pickle the outputs and just unpickle them each time we re-open the notebook
import pickle


Data is from: https://www.kaggle.com/datasets/toygarr/datasets-for-natural-language-processing

Dataset is from a collection of sentiment datasets, but we just want to play with the food one for now.

"ctweet, stweet, food" datasets are positive or negative analysis (sentiment) -> 0 negative -> 1 positive (ctweet has neutral 0, 1, 2)

we're assuming that any code we write to deal with this food data set will be extensible later by simply adding the rest of the data if we so choose. At the moment it's simply faster to work with a smaller subset of the data as we design the pipeline and it's displays.

## Utilities

In [166]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk import ngrams
import matplotlib.pyplot as plt

## Corpus and Vectoring tools

In [167]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
from gensim.models.ldamodel import LdaModel
from gensim.corpora.dictionary import Dictionary
import os

In [168]:
from sklearn.utils import resample

# Load the data

In [169]:
test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Sentiment-Capstone/data/food/test.csv')
train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Sentiment-Capstone/data/food/train.csv')
frames = [train, test]
df = pd.concat(frames)

Nobs to play with, want 'em nearby

In [122]:
sample_size = 50000
num_feats =10

# Sampling 

Here we can set up the code to downsample for speed's sake

In [123]:
#df = df.head(2000)

In [124]:
sample_size = 50000
df = resample(df,
                         replace=False,
                         n_samples=sample_size,
                        random_state=12,
                        )

In [125]:
df.shape

(50000, 2)

In [126]:
df.Y.value_counts()

1    42229
0     7771
Name: Y, dtype: int64

# Processing the text

### Tokenize text

In [170]:
df['tokenized'] = df.text.apply(word_tokenize)



In [171]:
df.head(2)

,text,Y,tokenized
0,i was put off at first by the green powder but...,1,"[i, was, put, off, at, first, by, the, green, ..."
1,these ginger chews are too good to be true i t...,1,"[these, ginger, chews, are, too, good, to, be,..."


### Remove Stop Words


In [172]:
stopwords = nltk.corpus.stopwords.words("english")

In [173]:
df['nostops'] = df.tokenized.apply(lambda  x: [item for item in x if item not in stopwords])

In [174]:
df.head(2)

,text,Y,tokenized,nostops
0,i was put off at first by the green powder but...,1,"[i, was, put, off, at, first, by, the, green, ...","[put, first, green, powder, bad, little, grain..."
1,these ginger chews are too good to be true i t...,1,"[these, ginger, chews, are, too, good, to, be,...","[ginger, chews, good, true, try, limit, one, d..."


### Stemming 
Might do lemmatization later, but stemming is simpler and more reliable and we're looking to get something working before we refine it. Lemmatization is a step we can experiment with in our refinement stages

We chose snowball because it's an older, and stable stemmer that incorporates improvements from the older stemmer algorithm Porter's real world experience
Snowball docs: https://www.nltk.org/api/nltk.stem.snowball.html
Article that informed our decision to choose Snowball: https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8


It looks as if stemming can increase recall, even in short texts but can also cause problems. We're going to move forward with stemmed words for now, but again can return to this if we seek optimization tasks down the line
Source: https://stackoverflow.com/questions/47219389/compute-word-n-grams-on-original-text-or-after-lemma-stemming-process#:~:text=Computing%20word%20n%2Dgrams%20after,you%20want%20to%20do%20it.

In [132]:
stemmer = SnowballStemmer("english")
df['stemmed'] = df.nostops.apply(lambda x: [stemmer.stem(item) for item in x])

In [134]:
df.stemmed.head(2)

33425     [like, low, carb, browni, top, notch, sever, b...
174759    [use, feed, dog, read, ingredi, despit, promot...
Name: stemmed, dtype: object

# Vectorizing the corpus
#Creating a df for each degree of n-gram vector, for easy organization
https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

### Count vectorizer Paramters to play with

max_dffloat in range [0.0, 1.0] or int, default=1.0
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

min_dffloat in range [0.0, 1.0] or int, default=1
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

max_featuresint, default=None
If not None, build a vocabulary that only consider the top max_features ordered by term frequency across the corpus.

This parameter is ignored if vocabulary is not None.

In [135]:
Vectorizer_Agent_mono = TfidfVectorizer(preprocessor=None, stop_words=stopwords, max_features=num_feats)

In [136]:
vectored_no_stops_mono = Vectorizer_Agent_mono.fit_transform(df.text.values) 

In [137]:
vectored_no_stops_mono.shape

(50000, 10)

### Bigram Vector

In [138]:
Vectorizer_Agent_bi = TfidfVectorizer(preprocessor=None, stop_words=stopwords, ngram_range=(1,2), max_features=num_feats)

In [139]:
vectored_no_stops_bi = Vectorizer_Agent_bi.fit_transform(df.text.values) 

In [140]:
vectored_no_stops_bi.shape

(50000, 10)

### Trigram Vector

In [141]:
Vectorizer_Agent_tri = TfidfVectorizer(preprocessor=None, stop_words=stopwords, ngram_range=(1,3), max_features=num_feats)

In [142]:
vectored_no_stops_tri = Vectorizer_Agent_tri.fit_transform(df.text.values) 

In [143]:
vectored_no_stops_tri.shape

(50000, 10)

# <span style="color:purple"> Clustering and Secondary EDA/Feature Engineering</span>

We played briefly with using DBSCAN, but it didn't converge at all, it gave us 360k categories

Thinking that instead of conventional clustering we're going to play with latent dirichlet allocation because it'll do a similar thing as K-means, but also provide us with a lot more information about the composition of those clusters.
Original Paper: https://ai.stanford.edu/~ang/papers/nips01-lda.pdf
Tutorial we used to help us get going: https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
Docs for the gensim module we're sdoing the lda with: https://radimrehurek.com/gensim/models/ldamodel.html

In [144]:
stemmed_texts = list(df.stemmed)

In [145]:
#makes a dictionary mapping to unique IDs
id2word = Dictionary(stemmed_texts)


In [146]:
#makes a bag of words version of the Gensim Dictionary
corpus = [id2word.doc2bow(text) for text in stemmed_texts]


In [147]:
# # Train the model on the corpus.
start = time.time()
num_topics = 10
lda = LdaModel(corpus,  id2word=id2word, num_topics=num_topics)
stop = time.time()
print("this took " +str(stop-start) + " seconds to complete")

this took 26.939680099487305 seconds to complete


# Feature Engineering with LDA

## Topic Vectors

Documentation we're going to need to use to get at document relationships to topics: https://radimrehurek.com/gensim/models/ldamodel.html

Relevant stack overflow
https://stackoverflow.com/questions/43357247/get-document-topics-and-get-term-topics-in-gensim

In [148]:
import gensim

In [149]:
start = time.time()
#Stole the below code from
#https://stackoverflow.com/questions/46574720/python-gensim-lda-add-the-topic-to-the-document-after-getting-the-topics
all_topics = lda.get_document_topics(corpus, minimum_probability=0.0)
all_topics_csr = gensim.matutils.corpus2csc(all_topics)
all_topics_numpy = all_topics_csr.T.toarray()
all_topics_df = pd.DataFrame(all_topics_numpy)
stop = time.time()
print("this took " +str(stop-start) + " seconds to complete")

this took 21.358879566192627 seconds to complete


In [150]:
#This is a feature of designing our code with downsampled data. Just need the index's to be consistent.
all_topics_df.index = df.index

In [151]:
#Let's rename the topic columns so that we can identify them later on
for each in all_topics_df.columns:
    all_topics_df['topic_' + str(each)] = all_topics_df[each]
    all_topics_df.drop(columns=[each], inplace=True)

In [152]:
all_topics_df

,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9
33425,0.003228,0.003228,0.003228,0.275443,0.117731,0.003228,0.189158,0.398302,0.003228,0.003228
174759,0.003705,0.003705,0.003705,0.003705,0.966654,0.003705,0.003705,0.003705,0.003705,0.003705
111229,0.004001,0.004002,0.004001,0.004001,0.963987,0.004001,0.004002,0.004001,0.004002,0.004002
209101,0.004763,0.004764,0.004763,0.260149,0.004764,0.004764,0.382232,0.324274,0.004764,0.004764
109643,0.698323,0.009096,0.009094,0.009095,0.228922,0.009095,0.009093,0.009095,0.009093,0.009094
...,...,...,...,...,...,...,...,...,...,...
59447,0.297327,0.224064,0.001112,0.040608,0.001113,0.213282,0.054235,0.001112,0.032557,0.134589
97003,0.004349,0.570731,0.004349,0.004349,0.004349,0.004350,0.004349,0.300656,0.098169,0.004349
64465,0.002129,0.002129,0.002129,0.272367,0.174221,0.538511,0.002129,0.002129,0.002129,0.002129
91943,0.067618,0.049209,0.000273,0.024400,0.139197,0.201719,0.406452,0.077397,0.033461,0.000273


In [153]:
import scipy

In [154]:
topic_vector = scipy.sparse.csr_matrix(all_topics_df.values)


# Aggregating our X

In [155]:
vector_stack = [vectored_no_stops_mono, vectored_no_stops_bi, vectored_no_stops_tri]

In [156]:
bi_vector_stack = [vectored_no_stops_mono, vectored_no_stops_bi]

In [157]:
mono_vector_stack = [vectored_no_stops_mono]

In [158]:
mono_and_topics = [vectored_no_stops_mono, topic_vector]

In [159]:
X = hstack(mono_vector_stack)

## Bringing in and extracting our Y column 

In [115]:
test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Sentiment-Capstone/data/food/test.csv')
train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Sentiment-Capstone/data/food/train.csv')
frames = [train, test]
df = pd.concat(frames)


In [13]:
df = resample(df,
                         replace=False,
                         n_samples=int(sample_size),
                        random_state=12,
                        )

In [161]:
y = df.Y

In [ ]:
y.value_counts(normalized=True)

# Train Test Split

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [163]:
print(X_train.shape)
print(y_train.shape)

(37500, 10)
(37500,)


# Save our Xs and ys to pass onto the next notebook

In [164]:
pickle.dump(X_train, open("X_train", 'wb'))
pickle.dump(X_test, open("X_test", 'wb'))
pickle.dump(y_train, open("y_train", 'wb'))
pickle.dump(y_test, open("y_test", 'wb'))